## Get electrodes from certain tasks using Chang Lab Database queries ##

You can also query the database for subjects that performed a given task, and plot these electrodes on the warped brain.

In [3]:
import numpy as np
import sys
#sys.path.append('/Users/dlchang/img_pipe/')
import img_pipe
sys.path.append('/Users/liberty/dura_mount/data_store2/imaging/changrepo/python/database/changlabdb')

from changlabdb.backend import *

from img_pipe.plotting.ctmr_brain_plot import el_add

In [9]:
# Get all the blocks from a certain task
task_oi = 'VOT_ID'
task = Task.objects.get(task_uid = task_oi)
blocks = Block.objects.filter(task=task)
task_patients = np.unique([block.session.patient.patient_uid for block in blocks])

print task_patients

[u'EC113' u'EC128' u'EC137' u'EC50' u'EC55']


In [10]:
# Plot the atlas brain
template = 'cvs_avg35_inMNI152'
atlas_patient = img_pipe.freeCoG(subj = template, hem='lh',\
                                 subj_dir = '/Users/liberty/dura_mount/data_store2/imaging/subjects')

roi = atlas_patient.roi('pial', opacity=0.5)
mesh, points, mlab = atlas_patient.plot_brain(rois = [roi], showfig=False)

# Now get all the STG electrodes from the patients who did this task
elecs = []
for s in task_patients:
    print s
    patient = img_pipe.freeCoG(subj = s, hem = 'stereo',\
                               subj_dir = '/Users/liberty/dura_mount/data_store2/imaging/subjects')
    # Find the warped electrodes in STG
    elecs_warped = patient.get_elecs(elecfile_prefix='TDT_elecs_all_warped', roi='superiortemporal')
    #elecs_warped =  patient.get_elecs(elecfile_prefix='%s_to_cvs_avg35_inMNI152'%s)
    # If there are any electrodes that fit these criteria, add them to the elecs list
    if elecs_warped['elecmatrix'] != []:
        print elecs_warped['elecmatrix'].shape
        elecs.append(elecs_warped['elecmatrix'])

elecmatrix = np.concatenate(elecs, axis=0)
# Add all the electrodes to the surface mesh
el_add(elecmatrix)    

# Render the mayavi window
mlab.show()

EC113


TypeError: 'NoneType' object has no attribute '__getitem__'